# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [46]:
df = data
df2 = df.groupby(["CustomerID","ProductName"]).agg({"Quantity":"sum"})



## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df3 = df2.pivot_table(index = "ProductName", columns = "CustomerID", values = "Quantity", aggfunc = np.sum)


In [6]:
df3 = df3.fillna(0)
df3

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0    0.0    0.0   
Apricots - Dried                     1.0    0.0    0.0    0.0    1.0    0.0   
...                                  ...    ...    ...    ...    ...    ...   
Yeast Dry - Fermipan                 0.0    0.0    0.0    0.0    0.0    0.0   
Yoghurt Tubes                        0.0    0.0    0.0    0.0    0.0    0.0   
Yogurt - Blueberry, 175 Gr           0.0    1.0    0.0    0.0    0.0    0.0   
Yogurt - French Vanilla              1.0    0.0    0.0    1.0    0.0    0.0   
Zucchini - Yellow                    0.0    0.0    0.0    0.0    0.0    0.0   

CustomerID                         477    639    649    669    ...  97697  \
ProductName                                                    ...          
Anchovy Paste - 56 G Tube            0.0    1.0    0.0    0.0  ...    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0  ...   25.0   
Appetizer - Mushroom Tart            0.0    1.0    0.0    0.0  ...   25.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0  ...    0.0   
Apricots - Dried                     0.0    0.0    0.0    0.0  ...    0.0   
...                                  ...    ...    ...    ...  ...    ...   
Yeast Dry - Fermipan                 0.0    0.0    0.0    0.0  ...    0.0   
Yoghurt Tubes                        0.0    0.0    2.0    0.0  ...    0.0   
Yogurt - Blueberry, 175 Gr           0.0    0.0    0.0    0.0  ...   25.0   
Yogurt - French Vanilla              2.0    0.0    0.0    1.0  ...    0.0   
Zucchini - Yellow                    0.0    0.0    0.0    0.0  ...    0.0   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube           25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp   25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0   25.0   25.0   25.0   
Apricots - Dried                    25.0    0.0    0.0    0.0    0.0    0.0   
...                                  ...    ...    ...    ...    ...    ...   
Yeast Dry - Fermipan                 0.0    0.0   25.0    0.0    0.0    0.0   
Yoghurt Tubes                        0.0    0.0    0.0    0.0    0.0    0.0   
Yogurt - Blueberry, 175 Gr           0.0    0.0   25.0    0.0    0.0    0.0   
Yogurt - French Vanilla              0.0   25.0    0.0    0.0    0.0    0.0   
Zucchini - Yellow                    0.0    0.0    0.0    0.0    0.0    0.0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube            0.0    0.0    0.0  
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0  
Appetizer - Mushroom Tart            0.0   25.0    0.0  
Appetizer - Sausage Rolls            0.0   25.0    0.0  
Apricots - Dried                     0.0    0.0    0.0  
...                                  ...    ...    ...  
Yeast Dry - Fermipan                 0.0    0.0    0.0  
Yoghurt Tubes                       25.0    0.0    0.0  
Yogurt - Blueberry, 175 Gr           0.0    0.0    0.0  
Yogurt - French Vanilla              0.0    0.0   25.0  
Zucchini - Yellow                    0.0    0.0    0.0  

[452 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [81]:
sm = squareform(pdist(df3.T, 'euclidean'))
sm = pd.DataFrame(sm)


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df3.T, 'euclidean'))), 
                         index=df3.columns, columns=df3.columns)

In [100]:
most_similar = list(distances[412].nlargest(6).index)
most_similar

[412, 3317, 3535, 1008, 1072, 639]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [71]:
df5 = pd.DataFrame()
for i in df2.index:
    if i[0] in most_similar:
        df5 = df5.append(df2.loc[i])


In [54]:


tuples = list(df5.index)

In [68]:
index = pd.MultiIndex.from_tuples(tuples, names=['CustomerID', 'ProductName'])

In [72]:
top5 = pd.DataFrame(df5["Quantity"],index = index)


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [85]:
prod_count = top5.groupby("ProductName").agg({"Quantity":"sum"})
prod_count = prod_count.sort_values("Quantity", ascending = False)

In [86]:
prod_count.head()

Quantity
ProductName                           
Spinach - Baby                     3.0
Soup - Campbells Bean Medley       3.0
Curry Paste - Madras               3.0
Pepper - Black, Whole              3.0
Soupfoamcont12oz 112con            3.0

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [94]:
prod_count = prod_count.reset_index()
df3 = df3.reset_index()
filtered = prod_count.merge(df3,on = "ProductName")


ValueError: cannot insert level_0, already exists

In [99]:
filt2 = filtered[["ProductName","Quantity",412]]
filt2["Difference"] = filtered["Quantity"] - filtered[412]
filt2.sort_values("Difference", ascending = False).head()[["ProductName","Difference"]]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ProductName  Difference
1    Soup - Campbells Bean Medley         3.0
3           Pepper - Black, Whole         3.0
5   Wine - Blue Nun Qualitatswein         3.0
57        Olive - Spread Tapenade         2.0
37                Fondant - Icing         2.0

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

150

In [128]:
custIDs = list(distances.index[:150])
custdict = {}
for ID in custIDs:
    most_similar = list(distances[ID].nlargest(6).index)
    most_similar.remove(ID)
    
    df5 = pd.DataFrame()
    for i in df2.index:
        if i[0] in most_similar:
            df5 = df5.append(df2.loc[i])
            
    tuples = list(df5.index)
    index = pd.MultiIndex.from_tuples(tuples, names=['CustomerID', 'ProductName'])
    top5 = pd.DataFrame(df5["Quantity"],index = index)
    
    prod_count = top5.groupby("ProductName").agg({"Quantity":"sum"})
    prod_count = prod_count.sort_values("Quantity", ascending = False)
    prod_count = prod_count.reset_index()
    filtered = prod_count.merge(df3,on = "ProductName")
    filt2 = filtered[["ProductName","Quantity",412]]
    filt2["Difference"] = filtered["Quantity"] - filtered[412]
    filt2 = filt2.sort_values("Difference", ascending = False).head()[["ProductName","Difference"]]

    custdict[ID] = list(filt2["ProductName"].values)
print("done")

    
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


done


{33: ['Butter - Unsalted',
  'Towels - Paper / Kraft',
  'Soup - Campbells Bean Medley',
  'Wine - Blue Nun Qualitatswein',
  'Pepper - Black, Whole'],
 200: ['Soup - Campbells Bean Medley',
  'Bay Leaf',
  'Pork - Kidney',
  'Pepper - Black, Whole',
  'Muffin - Carrot Individual Wrap'],
 264: ['Fondant - Icing',
  'Bread - Italian Roll With Herbs',
  'Wine - Blue Nun Qualitatswein',
  'Lettuce - Spring Mix',
  'Cocoa Butter'],
 356: ['Butter - Unsalted',
  'Beets - Candy Cane, Organic',
  'Veal - Inside, Choice',
  'Tia Maria',
  'Nut - Chestnuts, Whole'],
 412: ['Olive - Spread Tapenade',
  'Wine - Blue Nun Qualitatswein',
  'Sprouts - Baby Pea Tendrils',
  'Pepper - Black, Whole',
  'Soup - Campbells Bean Medley'],
 464: ['Butter - Unsalted',
  'Sauce - Gravy, Au Jus, Mix',
  'Yoghurt Tubes',
  'Ketchup - Tomato',
  'Nut - Chestnuts, Whole'],
 477: ['Olive - Spread Tapenade',
  'Wine - Blue Nun Qualitatswein',
  'Sprouts - Baby Pea Tendrils',
  'Pepper - Black, Whole',
  'Soup - Cam

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
#

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.